In [146]:
import os
import numpy as np
from PIL import Image

dataset_path = r"C:\Users\asus\Documents\MANAS\train\train"
input_size = 28 * 28
hidden_size = 64
output_size = 10

class_folders = sorted(os.listdir(dataset_path))
class_mapping = {folder: idx for idx, folder in enumerate(class_folders)}
num_classes = len(class_mapping)

all_image_data = []
all_labels = []
label_mapping = {}

for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    class_label = class_mapping[folder]

    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)

        image = Image.open(image_path)
        image_data = np.array(image).flatten() / 255.0
        if folder not in label_mapping:
            label_mapping[folder] = len(label_mapping)
        label = label_mapping[folder]

        label_one_hot = np.zeros(num_classes, dtype=int)
        label_one_hot[class_label] = 1
        all_image_data.append(image_data)
        all_labels.append(label_one_hot)

all_image_data = np.array(all_image_data)
all_labels = np.array(all_labels)
all_image_data = all_image_data.T
all_labels=all_labels.T

print("All image data shape:", all_image_data.shape)
print("All labels shape:", all_labels.shape)

m = all_image_data.shape[0]


All image data shape: (784, 8000)
All labels shape: (10, 8000)


In [147]:
def init_params():
    W1 = np.random.randn(10, 784)
    b1 = 0
    W2 = np.random.randn(10, 10)
    b2 = 0
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0


def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    dZ2 = A2 - Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

W1, b1, W2, b2 = gradient_descent(all_image_data, all_labels, 0.10, 500)

Iteration:  0
[1 5 1 ... 1 5 5] [[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
0.052975
Iteration:  10
[0 0 0 ... 0 0 0] [[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
0.8973
Iteration:  20
[0 0 0 ... 0 0 0] [[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
0.8985375
Iteration:  30
[0 0 0 ... 0 0 0] [[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
0.89865
Iteration:  40
[0 0 0 ... 0 0 0] [[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
0.8991
Iteration:  50
[0 0 0 ... 0 0 0] [[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
0.8992125
Iteration:  60
[0 0 0 ... 0 0 0]

In [151]:

num_test_images_per_class = 200
total_test_images = num_test_images_per_class * num_classes

test_dataset_path = r"C:\Users\asus\Documents\MANAS\test\test"

test_image_data = []
test_labels = []

for folder in os.listdir(test_dataset_path):
    folder_path = os.path.join(test_dataset_path, folder)
    class_label = class_mapping[folder]

    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)

        image = Image.open(image_path)
        image_data = np.array(image).flatten() / 255.0
        label_one_hot = np.zeros(num_classes, dtype=int)
        label_one_hot[class_label] = 1
        test_image_data.append(image_data)
        test_labels.append(label_one_hot)

test_image_data = np.array(test_image_data).T
test_labels = np.array(test_labels).T

print("Test image data shape:", test_image_data.shape)
print("Test labels shape:", test_labels.shape)

def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions
def get_accuracy(predictions, Y):
    predicted_classes = np.argmax(predictions, axis=0)
    true_classes = np.argmax(Y, axis=0)
    return np.sum(predicted_classes == true_classes) / Y.shape[1]

dev_predictions = make_predictions(test_image_data, W1, b1, W2, b2)
get_accuracy(dev_predictions, test_labels)



print("Test Accuracy:", test_accuracy)


Test image data shape: (784, 2000)
Test labels shape: (10, 2000)
Test Accuracy: 0.0
